In [ ]:
import pandas as pd
import numpy as np
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from pandas.plotting import table 
#!pip install -U kaleido
import plotly.figure_factory as ff

In [ ]:
username='neendoniss'
api_key=''
chart_studio.tools.set_credentials_file(username=username,
                                        api_key=api_key)

In [ ]:
#read incarceration data table from ICPSR download
pris_stat_df = pd.read_table('/work/38555/38555-0001-Data.tsv')

In [ ]:
#select year 2021 and relevant columns from incarceration data
rel_cols = ['YEAR','STATE','CUSTOTM','CUSTOTF','WHITEM','WHITEF','BLACKM','BLACKF','HISPM','HISPF','AIANM','AIANF','ASIANM','ASIANF','NHPIM','NHPIF']
pris_race_2021_df = pris_stat_df.loc[pris_stat_df['YEAR']==2021]
pris_race_2021_df = pris_race_2021_df[rel_cols]
pris_race_2021_df.drop(pris_race_2021_df[pris_race_2021_df['STATE'] == 'FE'].index, inplace = True)
pris_race_2021_df.drop(pris_race_2021_df[pris_race_2021_df['STATE'] == 'DC'].index, inplace=True)
pris_race_2021_df

,YEAR,STATE,CUSTOTM,CUSTOTF,WHITEM,WHITEF,BLACKM,BLACKF,HISPM,HISPF,AIANM,AIANF,ASIANM,ASIANF,NHPIM,NHPIF
2322,2021,AL,18181,1144,9897,1573,12816,537,0,0,3,0,4,0,0,0
2323,2021,AK,3992,417,1692,203,441,24,111,14,1679,176,143,15,90,5
2324,2021,AZ,23373,3114,11391,1569,4836,251,12231,900,1611,269,136,21,0,0
2325,2021,AR,13316,1179,8469,1078,6465,306,533,30,37,12,45,3,18,1
2326,2021,CA,95890,3839,18941,1196,27702,1008,44589,1410,1090,71,1091,71,307,14
2327,2021,CO,11860,1241,5940,580,2653,116,4394,328,490,52,168,16,9,1
2328,2021,CT,8834,588,2464,314,4036,172,2684,140,31,3,39,4,0,0
2329,2021,DE,4293,227,1427,124,2844,108,289,10,1,0,6,0,0,0
2331,2021,FL,63744,3391,28436,3163,36810,1456,9810,364,75,13,14,4,6,1
2332,2021,GA,36523,3068,15342,1817,26522,1193,1897,48,20,2,149,8,0,0


In [ ]:
#the codebook indicates that a -8 in columns with a count of prisoners means there was no data collected for that race
#count the number of -8
#note that in 2021, all states recorded number of prisoners for all races
pris_race_2021_df = pris_race_2021_df.replace(-8,np.nan)
pris_race_2021_df.isna().sum()

YEAR       0
STATE      0
CUSTOTM    0
CUSTOTF    0
WHITEM     0
WHITEF     0
BLACKM     0
BLACKF     0
HISPM      0
HISPF      0
AIANM      0
AIANF      0
ASIANM     0
ASIANF     0
NHPIM      0
NHPIF      0
dtype: int64

In [ ]:
print(pris_stat_df)
pris_stat_df = pris_stat_df[rel_cols].replace(-1,np.nan)
pris_stat_df.isna().sum()

      YEAR  STATEID STATE  REGION  CUSGT1M  CUSGT1F  CUSLT1M  CUSLT1F  \
0     1978        1    AL       3       -2       -2       -2       -2   
1     1978        2    AK       4       -2       -2       -2       -2   
2     1978        4    AZ       4       -2       -2       -2       -2   
3     1978        5    AR       3       -2       -2       -2       -2   
4     1978        6    CA       4       -2       -2       -2       -2   
...    ...      ...   ...     ...      ...      ...      ...      ...   
2371  2021       55    WI       2    17572     1159     1134       75   
2372  2021       56    WY       4     1605      248        0        0   
2373  2021       60    ST       7   822752    58886     8788     1533   
2374  2021       70    US       5   938335    66837    10746     1812   
2375  2021       99    FE       6   115583     7951     1958      279   

      CUSUNSM  CUSUNSF  ...  DTHHOMOM  DTHHOMOF  DTHPERSM  DTHPERSF  DTHOTHM  \
0          -2       -2  ...        -1      

YEAR          0
STATE         0
CUSTOTM       0
CUSTOTF       0
WHITEM        0
WHITEF        0
BLACKM        0
BLACKF        0
HISPM         0
HISPF         0
AIANM         0
AIANF         0
ASIANM     1026
ASIANF     1026
NHPIM      1026
NHPIF      1026
dtype: int64

In [ ]:
#add columns with total number of prisoners in each state
#and add columns with race as percent of prisoners in each state
#select relevant columns
pris_race_2021_df['total_prisoners'] = pris_race_2021_df['CUSTOTM'] + pris_race_2021_df['CUSTOTF']
pris_race_2021_df['white_prisoners'] = pris_race_2021_df['WHITEM'] + pris_race_2021_df['WHITEF']
pris_race_2021_df['white_%_of_prisoners'] = pris_race_2021_df['white_prisoners'] / pris_race_2021_df['total_prisoners']
pris_race_2021_df['black_prisoners'] = pris_race_2021_df['BLACKM'] + pris_race_2021_df['BLACKF']
pris_race_2021_df['black_%_of_prisoners'] = pris_race_2021_df['black_prisoners'] / pris_race_2021_df['total_prisoners']
pris_race_2021_df['hispanic_prisoners'] = pris_race_2021_df['HISPM'] + pris_race_2021_df['HISPF']
pris_race_2021_df['hispanic_%_of_prisoners'] = pris_race_2021_df['hispanic_prisoners'] / pris_race_2021_df['total_prisoners']
pris_race_2021_df['aian_prisoners'] = pris_race_2021_df['AIANM'] + pris_race_2021_df['AIANF']
pris_race_2021_df['aian_%_of_prisoners'] = pris_race_2021_df['aian_prisoners'] / pris_race_2021_df['total_prisoners']
pris_race_2021_df['asian_prisoners'] = pris_race_2021_df['ASIANM'] + pris_race_2021_df['ASIANF']
pris_race_2021_df['asian_%_of_prisoners'] = pris_race_2021_df['asian_prisoners'] / pris_race_2021_df['total_prisoners']
pris_race_2021_df['nhpi_prisoners'] = pris_race_2021_df['NHPIM'] + pris_race_2021_df['NHPIF']
pris_race_2021_df['nhpi_%_of_prisoners'] = pris_race_2021_df['nhpi_prisoners'] / pris_race_2021_df['total_prisoners']
pris_race_2021_df.columns

pris_race_2021_df = pris_race_2021_df[['STATE','total_prisoners', 'white_prisoners', 'white_%_of_prisoners',
       'black_prisoners', 'black_%_of_prisoners', 'hispanic_prisoners',
       'hispanic_%_of_prisoners', 'aian_prisoners', 'aian_%_of_prisoners',
       'asian_prisoners', 'asian_%_of_prisoners', 'nhpi_prisoners',
       'nhpi_%_of_prisoners']]

In [ ]:
pris_race_2021_df

,STATE,total_prisoners,white_prisoners,white_%_of_prisoners,black_prisoners,black_%_of_prisoners,hispanic_prisoners,hispanic_%_of_prisoners,aian_prisoners,aian_%_of_prisoners,asian_prisoners,asian_%_of_prisoners,nhpi_prisoners,nhpi_%_of_prisoners
2322,AL,19325,11470,0.593532,13353,0.690970,0,0.000000,3,0.000155,4,0.000207,0,0.000000
2323,AK,4409,1895,0.429803,465,0.105466,125,0.028351,1855,0.420730,158,0.035836,95,0.021547
2324,AZ,26487,12960,0.489297,5087,0.192056,13131,0.495753,1880,0.070978,157,0.005927,0,0.000000
2325,AR,14495,9547,0.658641,6771,0.467127,563,0.038841,49,0.003380,48,0.003311,19,0.001311
2326,CA,99729,20137,0.201917,28710,0.287880,45999,0.461240,1161,0.011642,1162,0.011652,321,0.003219
2327,CO,13101,6520,0.497672,2769,0.211358,4722,0.360431,542,0.041371,184,0.014045,10,0.000763
2328,CT,9422,2778,0.294842,4208,0.446614,2824,0.299724,34,0.003609,43,0.004564,0,0.000000
2329,DE,4520,1551,0.343142,2952,0.653097,299,0.066150,1,0.000221,6,0.001327,0,0.000000
2331,FL,67135,31599,0.470678,38266,0.569986,10174,0.151545,88,0.001311,18,0.000268,7,0.000104
2332,GA,39591,17159,0.433407,27715,0.700033,1945,0.049127,22,0.000556,157,0.003966,0,0.000000


In [ ]:
#read census poverty data table into df
#select relevant columns for reshaping
pov_df = pd.read_excel('ACSST1Y2021.S1701-2023-07-12T002713.xlsx',sheet_name='Data',index_col=[0],thousands=',')
pov_df = pov_df[pov_df.columns[::2]]
places = pov_df.columns[::3]
pov_df = pov_df.rename(index=str)
characteristics = ['Population for whom poverty status is determined','White alone','Black or African American alone',
                    'American Indian and Alaska Native alone','Asian alone','Native Hawaiian and Other Pacific Islander alone',
                    'Hispanic or Latino origin (of any race)']
pov_df = pov_df[pov_df.index.isin(characteristics)]
pov_df = pov_df.replace('N',np.nan)
pov_df

,United States,Unnamed: 3,Unnamed: 5,Alabama,Unnamed: 9,Unnamed: 11,Alaska,Unnamed: 15,Unnamed: 17,Arizona,...,Unnamed: 299,Wisconsin,Unnamed: 303,Unnamed: 305,Wyoming,Unnamed: 309,Unnamed: 311,Puerto Rico,Unnamed: 315,Unnamed: 317
Population for whom poverty status is determined,324173084,41393176,12.8%,4920613,794326,16.1%,716769,75165,10.5%,7126930,...,16.8%,5754788,621125,10.8%,565760,64224,11.4%,3235303,1310464,40.5%
White alone,198511109,19510415,9.8%,3218517,378269,11.8%,425946,32397,7.6%,4156705,...,16.1%,4629946,395036,8.5%,478359,48853,10.2%,904157,376501,41.6%
Black or African American alone,38536055,8402643,21.8%,1260356,327284,26.0%,19749,3298,16.7%,309600,...,28.3%,341604,100620,29.5%,6187,3283,53.1%,199489,79371,39.8%
American Indian and Alaska Native alone,3066848,656007,21.4%,24398,6205,25.4%,97879,21461,21.9%,285686,...,NaN,40421,7752,19.2%,10047,3452,34.4%,NaN,NaN,NaN
Asian alone,18833840,1918933,10.2%,67956,7852,11.6%,47028,3723,7.9%,242930,...,7.6%,172128,22786,13.2%,4916,938,19.1%,NaN,NaN,NaN
Native Hawaiian and Other Pacific Islander alone,594585,104847,17.6%,NaN,NaN,NaN,NaN,NaN,NaN,12221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hispanic or Latino origin (of any race),61398400,10759082,17.5%,230804,56551,24.5%,51016,4467,8.8%,2308717,...,20.1%,430388,77522,18.0%,60022,9428,15.7%,3211194,1302659,40.6%


In [ ]:
#borrowed dict for converting locations to 2 letter codes from https://gist.github.com/rogerallen/1583593
us_state_to_abbrev = {
    "United States": "US",
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "District of Columbia": "DC",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "Puerto Rico": "PR"
}

In [ ]:
#get indexes for columns with locations and rename to 2 letter codes
place_indices = [i for i in range(0,len(pov_df.columns),3)]
pov_df.rename(columns=us_state_to_abbrev, inplace=True)

In [ ]:
#rename columns from census tables so they are usable
below_poverty_level_indices = [i for i in range(1,len(pov_df.columns),3)]
percent_below_poverty_indices = [i for i in range(2,len(pov_df.columns),3)]

pov_df.rename(columns={pov_df.columns[i]:pov_df.columns[i-1]+'_bp' for i in below_poverty_level_indices}, inplace=True)
pov_df.rename(columns={pov_df.columns[i]:pov_df.columns[i-2]+'_bp%' for i in percent_below_poverty_indices}, inplace=True)

pov_df.index = ['total_pop','white','black','aian','asian','nhpi','hispanic']

pov_df

,US,US_bp,US_bp%,AL,AL_bp,AL_bp%,AK,AK_bp,AK_bp%,AZ,...,WV_bp%,WI,WI_bp,WI_bp%,WY,WY_bp,WY_bp%,PR,PR_bp,PR_bp%
total_pop,324173084,41393176,12.8%,4920613,794326,16.1%,716769,75165,10.5%,7126930,...,16.8%,5754788,621125,10.8%,565760,64224,11.4%,3235303,1310464,40.5%
white,198511109,19510415,9.8%,3218517,378269,11.8%,425946,32397,7.6%,4156705,...,16.1%,4629946,395036,8.5%,478359,48853,10.2%,904157,376501,41.6%
black,38536055,8402643,21.8%,1260356,327284,26.0%,19749,3298,16.7%,309600,...,28.3%,341604,100620,29.5%,6187,3283,53.1%,199489,79371,39.8%
aian,3066848,656007,21.4%,24398,6205,25.4%,97879,21461,21.9%,285686,...,NaN,40421,7752,19.2%,10047,3452,34.4%,NaN,NaN,NaN
asian,18833840,1918933,10.2%,67956,7852,11.6%,47028,3723,7.9%,242930,...,7.6%,172128,22786,13.2%,4916,938,19.1%,NaN,NaN,NaN
nhpi,594585,104847,17.6%,NaN,NaN,NaN,NaN,NaN,NaN,12221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hispanic,61398400,10759082,17.5%,230804,56551,24.5%,51016,4467,8.8%,2308717,...,20.1%,430388,77522,18.0%,60022,9428,15.7%,3211194,1302659,40.6%


In [ ]:
#rename and reshape with help from Stack Overflow
#https://stackoverflow.com/questions/76707900/is-it-even-possible-to-reshape-this-using-pandas/
col_pairs = pov_df.columns.str.extract(r'^([A-Z]{2})(?:_(.*))?$').fillna('pop')
multi = pd.MultiIndex.from_arrays(col_pairs.T.values)
pov_df.columns = multi
pov_df = pov_df.T.unstack(level=1)
pov_df

total_pop                       white                      black         \
          bp    bp%        pop        bp    bp%        pop       bp    bp%   
AK     75165  10.5%     716769     32397   7.6%     425946     3298  16.7%   
AL    794326  16.1%    4920613    378269  11.8%    3218517   327284  26.0%   
AR    480153  16.3%    2944289    276781  13.4%    2064398   123828  29.0%   
AZ    908961  12.8%    7126930    399147   9.6%    4156705    51696  16.7%   
CA   4733036  12.3%   38481790   1470782   9.7%   15154116   407932  19.9%   
CO    553272   9.7%    5696140    324117   8.1%    4024108    38372  17.7%   
CT    354166  10.1%    3506226    165247   7.2%    2294990    57579  15.8%   
DC    105007  16.5%     637491     12099   5.0%     239702    77030  27.7%   
DE    113450  11.6%     978675     46530   7.8%     597120    31882  16.1%   
FL   2805433  13.1%   21368535   1231762  10.3%   11992853   646127  20.4%   
GA   1476348  14.0%   10529506    532767   9.7%    5475713   648601  20.1%   
HI    156735  11.2%    1402392     36505  11.9%     305835     3723  14.3%   
IA    344696  11.1%    3100317    252010   9.6%    2620584    30528  28.3%   
ID    205702  11.0%    1868374    156002  10.2%    1522134     3529  27.6%   
IL   1498523  12.1%   12380582    658730   8.7%    7596663   408495  24.8%   
IN    803021  12.2%    6608427    529045  10.2%    5166265   117738  20.6%   
KS    333518  11.7%    2854624    216531  10.0%    2173206    33084  23.1%   
KY    721878  16.5%    4379845    561959  15.4%    3655817    73804  24.1%   
LA    883236  19.6%    4501071    325849  12.4%    2621687   440904  31.9%   
MA    700138  10.4%    6748410    363870   7.8%    4677813    67758  15.3%   
MD    618372  10.3%    6023847    198881   6.7%    2965003   251905  14.5%   
ME    154117  11.5%    1338019    136301  11.2%    1214188     5277  27.8%   
MI   1286329  13.1%    9847403    754078  10.3%    7304986   340973  26.2%   
MN    519731   9.3%    5587189    326270   7.5%    4350858    78080  20.9%   
MO    761311  12.7%    5999824    494099  10.6%    4650310   146290  22.9%   
MS    554152  19.4%    2861627    186362  11.5%    1617153   318594  31.2%   
MT    128123  11.9%    1079864     92813  10.1%     920920      NaN    NaN   
NC   1378621  13.4%   10285053    627055   9.7%    6438600   421381  20.6%   
ND     83350  11.1%     752033     57471   9.2%     626675     4632  21.3%   
NE    205852  10.8%    1914212    133545   8.9%    1504348    17696  20.6%   
NH     97403   7.2%    1348892     82260   6.9%    1189113     1188   6.4%   
NJ    930602  10.2%    9093135    343043   6.9%    4977018   187553  16.5%   
NM    382798  18.4%    2076524    133901  14.3%     938555     9924  24.8%   
NV    437385  14.1%    3103176    177193  11.5%    1540310    72100  25.7%   
NY   2688587  13.9%   19329338   1071233  10.0%   10754607   547919  20.3%   
OH   1536524  13.4%   11470517    943925  10.6%    8891543   369290  27.5%   
OK    606782  15.6%    3877776    337406  13.3%    2544799    67382  26.1%   
OR    507829  12.2%    4166362    354145  11.4%    3114538    15460  20.0%   
PA   1519032  12.1%   12571365    855801   9.1%    9435818   316173  24.8%   
PR   1310464  40.5%    3235303    376501  41.6%     904157    79371  39.8%   
RI    120055  11.4%    1052091     57318   7.7%     744026    10135  20.2%   
SC    741652  14.6%    5063148    326040  10.1%    3225889   299926  23.8%   
SD    106548  12.3%     867039     57431   8.2%     702298     3227  19.0%   
TN    927587  13.6%    6811613    557432  11.2%    4991973   231137  22.2%   
TX   4122538  14.2%   28933638   1447169  10.5%   13840114   660174  19.5%   
US  41393176  12.8%  324173084  19510415   9.8%  198511109  8402643  21.8%   
UT    281673   8.6%    3290444    204222   7.8%    2610568     7837  22.7%   
VA    854145  10.2%    8401227    402234   7.9%    5119659   249797  16.5%   
VT     63720  10.3%     621338     54848   9.7%     567796     1404  23.0%   
WA    754315   9.9%    75

In [ ]:
#replace spaces with underscore in column names, reset index, and rename column to 'STATE'
pov_df.columns = pov_df.columns.map(lambda x: '_'.join([str(i) for i in x]))
pov_df.reset_index(inplace=True)
pov_df.rename(columns={'index':'STATE'}, inplace=True)

In [ ]:
#merge incarceration and poverty dfs into one
#census data had strings and symbols instead of floats or integers
#convert strings to floats
pris_race_pov_df = pov_df.merge(pris_race_2021_df,on='STATE')
for col in pris_race_pov_df.columns[1:]:
    if not col.endswith('%'):
        pris_race_pov_df[col] = pd.to_numeric(pris_race_pov_df[col], errors='coerce')
    elif col.endswith('%'):
        pris_race_pov_df[col].fillna(value='0%',inplace=True)
        pris_race_pov_df[col] = pris_race_pov_df[col].apply(lambda x: x.rstrip('%'))
        pris_race_pov_df[col] = pris_race_pov_df[col].apply(lambda x: float(x))
        pris_race_pov_df[col] = pris_race_pov_df[col].replace(0,np.nan)

#check to see how many states collected no data for different races
#note that 1 state had no data for the number of Black people below poverty
#note that 5 states had no data for the number of AIAN people below poverty
#note that 35 states had no data for the number of NHPI people below poverty
print(pris_race_pov_df.isna().sum())

#replace nan with 0
pris_race_pov_df.fillna(0,inplace=True)

STATE                       0
total_pop_bp                0
total_pop_bp%               0
total_pop_pop               0
white_bp                    0
white_bp%                   0
white_pop                   0
black_bp                    1
black_bp%                   1
black_pop                   1
aian_bp                     5
aian_bp%                    5
aian_pop                    5
asian_bp                    0
asian_bp%                   0
asian_pop                   0
nhpi_bp                    35
nhpi_bp%                   35
nhpi_pop                   35
hispanic_bp                 0
hispanic_bp%                0
hispanic_pop                0
total_prisoners             0
white_prisoners             0
white_%_of_prisoners        0
black_prisoners             0
black_%_of_prisoners        0
hispanic_prisoners          0
hispanic_%_of_prisoners     0
aian_prisoners              0
aian_%_of_prisoners         0
asian_prisoners             0
asian_%_of_prisoners        0
nhpi_priso

In [ ]:
#add columns with prisoners as percent of total population and by percent of each race
#add columns with percent of population for each race
#add columns with ratio of incarceration rate to each race's percent of population
pris_race_pov_df['white_prisoners_%_of_pop'] = pris_race_pov_df['white_prisoners']/pris_race_pov_df['total_pop_pop']*100
pris_race_pov_df['black_prisoners_%_of_pop'] = pris_race_pov_df['black_prisoners']/pris_race_pov_df['total_pop_pop']*100
pris_race_pov_df['aian_prisoners_%_of_pop'] = pris_race_pov_df['aian_prisoners']/pris_race_pov_df['total_pop_pop']*100
pris_race_pov_df['asian_prisoners_%_of_pop'] = pris_race_pov_df['asian_prisoners']/pris_race_pov_df['total_pop_pop']*100
pris_race_pov_df['nhpi_prisoners_%_of_pop'] = pris_race_pov_df['nhpi_prisoners']/pris_race_pov_df['total_pop_pop']*100
pris_race_pov_df['hispanic_prisoners_%_of_pop'] = pris_race_pov_df['hispanic_prisoners']/pris_race_pov_df['total_pop_pop']*100
pris_race_pov_df['white_incarc_rate'] = pris_race_pov_df['white_prisoners']/pris_race_pov_df['white_pop']*100
pris_race_pov_df['black_incarc_rate'] = pris_race_pov_df['black_prisoners']/pris_race_pov_df['black_pop']*100
pris_race_pov_df['aian_incarc_rate'] = pris_race_pov_df['aian_prisoners']/pris_race_pov_df['aian_pop']*100
pris_race_pov_df['asian_incarc_rate'] = pris_race_pov_df['asian_prisoners']/pris_race_pov_df['asian_pop']*100
pris_race_pov_df['nhpi_incarc_rate'] = pris_race_pov_df['nhpi_prisoners']/pris_race_pov_df['nhpi_pop']*100
pris_race_pov_df['hispanic_incarc_rate'] = pris_race_pov_df['hispanic_prisoners']/pris_race_pov_df['hispanic_pop']*100
pris_race_pov_df['white_%_of_pop'] = pris_race_pov_df['white_pop']/pris_race_pov_df['total_pop_pop']*100
pris_race_pov_df['black_%_of_pop'] = pris_race_pov_df['black_pop']/pris_race_pov_df['total_pop_pop']*100
pris_race_pov_df['aian_%_of_pop'] = pris_race_pov_df['aian_pop']/pris_race_pov_df['total_pop_pop']*100
pris_race_pov_df['asian_%_of_pop'] = pris_race_pov_df['asian_pop']/pris_race_pov_df['total_pop_pop']*100
pris_race_pov_df['nhpi_%_of_pop'] = pris_race_pov_df['nhpi_pop']/pris_race_pov_df['total_pop_pop']*100
pris_race_pov_df['hispanic_%_of_pop'] = pris_race_pov_df['hispanic_pop']/pris_race_pov_df['total_pop_pop']*100
pris_race_pov_df['total_prisoners_%_of_pop'] = pris_race_pov_df['total_prisoners']/pris_race_pov_df['total_pop_pop']*100
pris_race_pov_df['white_pris_pop_ratio'] = pris_race_pov_df['white_%_of_prisoners']/pris_race_pov_df['white_%_of_pop']
pris_race_pov_df['black_pris_pop_ratio'] = pris_race_pov_df['black_%_of_prisoners']/pris_race_pov_df['black_%_of_pop']
pris_race_pov_df['aian_pris_pop_ratio'] = pris_race_pov_df['aian_%_of_prisoners']/pris_race_pov_df['aian_%_of_pop']
pris_race_pov_df['asian_pris_pop_ratio'] = pris_race_pov_df['asian_%_of_prisoners']/pris_race_pov_df['asian_%_of_pop']
pris_race_pov_df['nhpi_pris_pop_ratio'] = pris_race_pov_df['nhpi_%_of_prisoners']/pris_race_pov_df['nhpi_%_of_pop']
pris_race_pov_df['hispanic_pris_pop_ratio'] = pris_race_pov_df['hispanic_%_of_prisoners']/pris_race_pov_df['hispanic_%_of_pop']
pris_race_pov_df.replace([np.nan,np.inf,0],[0,0,0],inplace=True)
pris_race_pov_df

,STATE,total_pop_bp,total_pop_bp%,total_pop_pop,white_bp,white_bp%,white_pop,black_bp,black_bp%,black_pop,...,asian_%_of_pop,nhpi_%_of_pop,hispanic_%_of_pop,total_prisoners_%_of_pop,white_pris_pop_ratio,black_pris_pop_ratio,aian_pris_pop_ratio,asian_pris_pop_ratio,nhpi_pris_pop_ratio,hispanic_pris_pop_ratio
0,AK,75165,10.5,716769,32397,7.6,425946,3298.0,16.7,19749.0,...,6.561110,0.000000,7.117495,0.615121,0.007233,0.038278,0.030810,0.005462,0.000000,0.003983
1,AL,794326,16.1,4920613,378269,11.8,3218517,327284.0,26.0,1260356.0,...,1.381047,0.000000,4.690554,0.392736,0.009074,0.026976,0.000313,0.000150,0.000000,0.000000
2,AR,480153,16.3,2944289,276781,13.4,2064398,123828.0,29.0,426337.0,...,1.462051,0.396428,8.276973,0.492309,0.009394,0.032260,0.006024,0.002265,0.003307,0.004693
3,AZ,908961,12.8,7126930,399147,9.6,4156705,51696.0,16.7,309600.0,...,3.408621,0.171476,32.394271,0.371647,0.008389,0.044211,0.017707,0.001739,0.000000,0.015304
4,CA,4733036,12.3,38481790,1470782,9.7,15154116,407932.0,19.9,2049591.0,...,15.312159,0.387014,40.201046,0.259159,0.005127,0.054051,0.008549,0.000761,0.008317,0.011473
5,CO,553272,9.7,5696140,324117,8.1,4024108,38372.0,17.7,217190.0,...,3.195778,0.000000,22.234338,0.229998,0.007045,0.055432,0.040685,0.004395,0.000000,0.016211
6,CT,354166,10.1,3506226,165247,7.2,2294990,57579.0,15.8,363349.0,...,4.756025,0.000000,17.750995,0.268722,0.004505,0.043097,0.010863,0.000960,0.000000,0.016885
7,DE,113450,11.6,978675,46530,7.8,597120,31882.0,16.1,198357.0,...,4.143459,0.000000,10.168851,0.461849,0.005624,0.032223,0.000000,0.000320,0.000000,0.006505
8,FL,2805433,13.1,21368535,1231762,10.3,11992853,646127.0,20.4,3171916.0,...,2.869471,0.057599,26.951623,0.314177,0.008386,0.038399,0.004553,0.000093,0.001810,0.005623
9,GA,1476348,14.0,10529506,532767,9.7,5475713,648601.0,20.1,3222546.0,...,4.298758,0.000000,10.060795,0.376001,0.008334,0.022873,0.001145,0.000922,0.000000,0.004883


In [ ]:
#separate into 1 df with just states and 1 df with whole U.S.
states_pris_race_pov_df = pris_race_pov_df[pris_race_pov_df['STATE'] != 'US']
us_pris_race_pov_df = pris_race_pov_df[pris_race_pov_df['STATE'] == 'US']

In [ ]:
races = pd.DataFrame({'white':'White',
    'black':'Black',
    'aian':'American Indian/Alaska Native',
    'asian': 'Asian',
    'nhpi':'Native Hawaiian/Pacific Islander',
    'hispanic':'Hispanic'}
    , index=[0])

def make_map_plots(df,col_name,color_title,plot_title):
    rows = 2
    cols = 3
    fig = make_subplots(
        rows=rows, cols=cols,
        specs = [[{'type':'choropleth'} for c in np.arange(cols)] for r in np.arange(rows)],
        subplot_titles = list(races.loc[0,:]))

    for i, race in enumerate(races):
        result = df[['STATE',race+col_name]].replace([np.nan,np.inf,0],[0,0,0])
        fig.add_trace(go.Choropleth(
            locations = result.STATE,
            z = result[race+col_name],
            locationmode = 'USA-states',
            marker_line_color = 'white',
            zmin = df.drop('STATE',axis=1).min().min(),
            zmax = df.drop('STATE',axis=1).max().max(),
            colorbar_title = color_title
        ), row = i//cols+1, col = i%cols+1)

    fig.update_layout(
        title_text = plot_title,
        **{'geo' + str(i) + '_scope': 'usa' for i in [''] + np.arange(2,rows*cols+1).tolist()}
        )

    for index, trace in enumerate(fig.data):
        fig.data[index].hovertemplate = 'State: %{location}<br>{color_title}: %{z:.2f}<extra></extra>'
    fig.show()
    #when uncommented, the code below saves the fig to my plotly dashboard
    #py.plot(fig, filename='fig'+col_name, auto_open = True)

In [ ]:
pris_rate_df = states_pris_race_pov_df[['STATE','white_incarc_rate',
        'black_incarc_rate',
        'aian_incarc_rate',
        'asian_incarc_rate',
        'nhpi_incarc_rate',
        'hispanic_incarc_rate']]
incar_ratio_pop_df = states_pris_race_pov_df[['STATE','white_%_of_prisoners',
        'black_%_of_prisoners',
        'aian_%_of_prisoners',
        'asian_%_of_prisoners',
        'nhpi_%_of_prisoners',
        'hispanic_%_of_prisoners']]
make_map_plots(pris_rate_df,'_incarc_rate','Incarceration Rate','Incarceration Rate by Race')
make_map_plots(incar_ratio_pop_df,'_%_of_prisoners','Ratio','Ratio by Race: Percent of Prison Population to Percent of Total Population')

In [ ]:
#simple linear regression to see how much influence poverty rates have on incarceration rates across states
#White people have the highest correlation between poverty rate and incarceration rate, and Asian people have the lowest
#Black, AIAN, and NHPI people all have similar correlations between poverty rate and incarceration rate
r_squared = []
rows = 3
cols = 2
fig = make_subplots(rows = rows, cols = cols,
        specs= [[{'type': 'scatter'} for c in np.arange(cols)] for r in np.arange(rows)],
        subplot_titles=list(races.loc[0,:]))
row_no = 0
col_no = 0
reg_df = states_pris_race_pov_df[['STATE','white_bp%','black_bp%','aian_bp%','asian_bp%','nhpi_bp%','hispanic_bp%','white_incarc_rate','black_incarc_rate','aian_incarc_rate','asian_incarc_rate','nhpi_incarc_rate','hispanic_incarc_rate']]
for i, race in enumerate(races):
    i += 1
    if i < 3:
        row_no = 1
        col_no = i
    if i > 2 and i < 5:
        row_no = 2
        col_no = i - 2
    if i > 4:
        row_no = 3
        col_no = i - 4

    model = LinearRegression()
    X = states_pris_race_pov_df[race+'_bp%']
    y = states_pris_race_pov_df[race+'_incarc_rate']
    X_train, X_test, y_train, y_test = train_test_split(np.array(X).reshape(-1,1), y, test_size=0.2, random_state=0)
    reg = model.fit(np.array(X).reshape(-1,1), y)
    y_pred = model.predict(np.array(X).reshape(-1,1))

    r_squared.append(r2_score(y,y_pred))

    fig.add_trace(go.Scatter(
        x = X,
        y = y,
        mode = 'markers'
    ), row = row_no, col = col_no)
    fig.add_trace(go.Scatter(
        x = X,
        y = y_pred,
        mode = 'lines'
    ), row = row_no, col = col_no)

fig.update_layout(title_text = 'Incarceration Rate versus Poverty Rate',showlegend = False)
fig.show()
#when uncommented, the code below saves the fig to my plotly dashboard
#py.plot(fig, filename='lin_reg', auto_open = True)

'https://plotly.com/~neendoniss/47/'

In [ ]:
r_squared_df = races.T.reset_index().drop('index',axis=1)
r_squared_df.columns=['race']
r_squared_df['r_squared'] = [round(i,4) for i in r_squared]
r_squared_df

,race,r_squared
0,White,0.3025
1,Black,0.1880
2,American Indian/Alaska Native,0.1702
3,Asian,0.0000
4,Native Hawaiian/Pacific Islander,0.2042
5,Hispanic,0.0002


In [ ]:
fig =  ff.create_table(r_squared_df)
fig.update_layout(
    autosize=False,
    width=500,
    height=200,
)
fig.write_image("table_plotly.png", scale=2)
fig.show()
#when uncommented, the code below saves the fig to my plotly dashboard
#py.plot(fig, file_name='r_squared', auto_open=True)

'https://plotly.com/~neendoniss/54/'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=df7d6d94-60e6-49b1-8250-e010091bce36' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>